In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

import sys
sys.path.append('/home/rustam/hyperspecter_segmentation/makitorch')
sys.path.append('/home/rustam/hyperspecter_segmentation/')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.ndimage import gaussian_filter
import cv2
from hsi_dataset_api import HsiDataset

In [ ]:
pca_explained_variance = np.load('../PcaExplainedVariance_.npy')
pca_mean = np.load('../PcaMean.npy')
pca_components = np.load('../PcaComponents.npy')

In [ ]:
def pca_transformation(x):
    x_t = np.reshape(x, (x.shape[0], -1)) # (C, H, W) -> (C, H * W)
    x_t = np.swapaxes(x_t, 0, 1) # (C, H * W) -> (H * W, C)
    x_t = x_t - pca_mean
    x_t = np.dot(x_t, pca_components.T) / np.sqrt(pca_explained_variance)
    return np.reshape(x_t, (x.shape[1], x.shape[2], pca_components.shape[0])).astype(np.float32) # (H, W, N)

In [ ]:
test_indices = np.load('../test_indices.npy')
train_indices = np.load('../train_indices.npy')
path = '/raid/rustam/hyperspectral_dataset/new_cropped_hsi_data/'

In [ ]:
dataset = HsiDataset(path)

In [ ]:
images = []
masks = []

for idx, data_point in enumerate(dataset.data_iterator(opened=True, shuffle=False)):
    if train_indices is not None and idx not in train_indices:
        continue
    images.append(data_point.hsi)
    masks.append(data_point.mask)

In [ ]:
class2count = dict([(f'{i}', 0) for i in range(17)])
for mask in masks:
    for i in range(17):
        if i in mask:
            class2count[str(i)] += 1
class2count

In [ ]:
test_class2count = class2count

In [ ]:
for key in class2count.keys():
    print(f"Class={key.zfill(2)} count_train={str(class2count[key]).zfill(3)} count_test={str(test_class2count[key]).zfill(2)}")

In [ ]:
find_class = 16

In [ ]:
found_indx = []
for m in range(len(masks)):
    if find_class in masks[m]:
        found_indx.append(m)
found_indx

In [ ]:
i = 0

In [ ]:
i +=1

In [ ]:
indx_taken = found_indx[i]

hyperspecter = images[indx_taken].copy()
h_pca = pca_transformation(hyperspecter)
mask = masks[indx_taken].copy()

In [ ]:
hyperspecter.shape, h_pca.shape, np.unique(mask)

In [ ]:
sns.heatmap(h_pca[..., 0])

In [ ]:
sns.heatmap(hyperspecter[120])

In [ ]:
normed_h = (hyperspecter - hyperspecter.min()) / (hyperspecter.max() - hyperspecter.min())

In [ ]:
sns.heatmap(normed_h[200])

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(mask[..., 0])

In [ ]:
mask_s = mask[..., 0].copy()

In [ ]:
from sklearn.metrics import jaccard_score

In [ ]:
def matrix2onehot(matrix, num_classes=17):
    matrix = matrix.copy().reshape(-1)
    one_hoted = np.zeros((matrix.size, num_classes-1))
    one_hoted[np.arange(matrix.size),matrix] = 1
    return one_hoted

In [ ]:
mask_s.shape

In [ ]:
one_hoted = matrix2onehot(mask_s)
res = jaccard_score(one_hoted, one_hoted, average=None, zero_division=1)
res, res.shape, res.mean()

In [ ]:
np.unique(mask_s)

In [ ]:
meta

In [ ]:
mask_filtered = gaussian_filter(mask[..., 0], 0.5)

In [ ]:
sns.heatmap(mask_filtered)

In [ ]:
mask_filtered = cv2.blur(mask[..., 0], (4,4))

In [ ]:
sns.heatmap(mask_filtered)

In [ ]:
mask_filtered = cv2.GaussianBlur(mask[..., 0],(3,3),3)

In [ ]:
sns.heatmap(mask_filtered)

In [ ]:
mask_filtered = cv2.medianBlur(mask[..., 0],11)

In [ ]:
sns.heatmap(mask_filtered)

In [ ]:
mask_filtered = cv2.bilateralFilter(mask[..., 0],15,5,2)

In [ ]:
sns.heatmap(mask_filtered)

In [ ]:
mask_filtered = cv2.bilateralFilter(mask[..., 0],15,5,5)

In [ ]:
sns.heatmap(mask_filtered)